# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 

for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
     
        for line in csvreader:
            full_data_rows_list.append(line) 

print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


### check the number of rows in your csv file

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

session.execute('''
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    ''')

session = cluster.connect('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [25]:
def execute_query(session, query, fetch=False):
    try:
        result = session.execute(query)
        if fetch:
            rows = list(result)
            if rows:
                for row in rows:
                    print(row)
            else:
                print("No results found")
        return result
    except Exception as e:
        print(f'Error executing query: {e}')
        raise

def insert_data(file_path, table, columns, data_mapper):
    with open(file_path, encoding='utf-8') as f:
        csvreader = csv.reader(f)
        next(csvreader)

        columns_list = [col.strip() for col in columns.split(',')]
        placeholders = ', '.join(['?'] * len(columns.split(',')))
        query = f'INSERT INTO {table} ({columns}) VALUES ({placeholders})'
        query = session.prepare(query)

        row_count = 0
        for line in csvreader:
            try:
                values = data_mapper(line)
                session.execute(query, values)
                row_count += 1
            except Exception as e:
                print(f'Error inserting row: {e}')
                continue

        print(f'Successfully inserted {row_count} rows into {table}')

file_path = 'event_datafile_new.csv'

# 1. Table Name: sessions_songs
**Purpose:**

This table is designed to answer Query 1
To retrieve the artist, song title and song's length based on sessionId and itemInSession

**Primary Key:**

* Partition Key: ```sessionId``` (Used to partition data by the session identifier)
* Clustering Key: ```itemInSession``` (Ensures uniqueness and orders songs within a session)

In [16]:
# TABLE 1: sessions_songs
query_sessions_songs = '''
    CREATE TABLE IF NOT EXISTS udacity.sessions_songs (
        sessionId     INT,
        itemInSession SMALLINT,
        artist        TEXT,
        length        DECIMAL,
        song          TEXT,
        PRIMARY KEY (sessionId, itemInSession)
    );
'''

execute_query(session=session, query=query_sessions_songs)

insert_data(
    file_path=file_path,
    table='udacity.sessions_songs',
    columns='sessionId, itemInSession, artist, length, song',
    data_mapper=lambda line: (
        int(line[8]),   # sessionId
        int(line[3]),   # itemInSession
        line[0],        # artist
        float(line[5]), # length
        line[9]         # song
        )
    )

Successfully inserted 6820 rows into udacity.sessions_songs


# 2. Table Name: user_sessions_songs
**Purpose:**

This table is designed to answer Query 2.
To retrieve the name of artist, song (sorted by itemInSession) and user (first and last name) for a specific userId and sessionId.

**Primary Key:**

* Partition Key: ```(userId, sessionId)``` (Composite partition key to distribute data by user and session combination).
* Clustering Key: ```itemInSession``` (Orders songs by their sequence in the session).

In [17]:
# TABLE 2: user_sessions_songs
query_user_sessions_songs = '''
    CREATE TABLE IF NOT EXISTS udacity.user_sessions_songs (
        userId        INT,
        sessionId     INT,
        itemInSession SMALLINT,
        artist        TEXT,
        song          TEXT,
        firstName     TEXT,
        lastName      TEXT,
        PRIMARY KEY ((userId, sessionId), itemInSession)
    );
'''

execute_query(session=session, query=query_user_sessions_songs)

insert_data(
    file_path=file_path,
    table='udacity.user_sessions_songs',
    columns='userId, sessionId, itemInSession, artist, song, firstName, lastName',
    data_mapper=lambda line: (
            int(line[10]), # userId
            int(line[8]),  # sessionId
            int(line[3]),  # itemInSession
            line[0],       # artist
            line[9],       # song
            line[1],       # firstName
            line[4]        # lastName
        )
    )

Successfully inserted 6820 rows into udacity.user_sessions_songs


# 3. Table Name: song_listeners
**Purpose:**

This table is designed to answer Query 3.
To retrieve every user name (first and last) who listened to a specific song.

**Primary Key:**

* Partition Key: ```song``` (Used to partition data by song title).
* Clustering Key: ```userId``` (Ensures uniqueness - one row per user per song).

In [18]:
# TABLE 3: song_listeners
query_song_listeners = '''
    CREATE TABLE IF NOT EXISTS udacity.song_listeners (
        song          TEXT,
        userId        INT,
        firstName     TEXT,
        lastName      TEXT,
        artist        TEXT,
        PRIMARY KEY (song, userId)
    );
    '''

execute_query(session=session, query=query_song_listeners)

insert_data(
    file_path=file_path,
    table='udacity.song_listeners',
    columns='userId, song, firstName, lastName, artist',
    data_mapper=lambda line: (
            int(line[10]), # userId
            line[9],       # song
            line[1],       # firstName
            line[4],       # lastName
            line[0]        # artist
        )
    )

Successfully inserted 6820 rows into udacity.song_listeners


#### Do a SELECT to verify that the data have been inserted into each table

In [26]:
print("\n=== Query 1: Sessions Songs ===")
execute_query(session=session, query='SELECT * FROM udacity.sessions_songs LIMIT 1', fetch=True)

print("\n=== Query 2: User Sessions Songs ===")
execute_query(session=session, query='SELECT * FROM udacity.user_sessions_songs LIMIT 1', fetch=True)

print("\n=== Query 3: Song Listeners ===")
execute_query(session=session, query='SELECT * FROM udacity.song_listeners LIMIT 1', fetch=True)


=== Query 1: Sessions Songs ===
Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=Decimal('191.08526000000000522049958817660808563232421875'), song='The Calculation (Album Version)')

=== Query 2: User Sessions Songs ===
Row(userid=58, sessionid=768, iteminsession=0, artist='System of a Down', firstname='Emily', lastname='Benson', song='Sad Statue')

=== Query 3: Song Listeners ===
Row(song="Wonder What's Next", userid=49, artist='Chevelle', firstname='Chloe', lastname='Cuevas')


## Create queries to ask the following three questions of the data



In [27]:
# 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
print("\n\n=== QUERY 1: Get artist, song title and length for sessionId=338, itemInSession=4 ===")
execute_query(
    session=session,
    query='''
        SELECT
            artist,
            song,
            length
        FROM
            udacity.sessions_songs
        WHERE
            sessionid = 338
            AND iteminsession = 4;
    ''',
    fetch=True
    )



=== QUERY 1: Get artist, song title and length for sessionId=338, itemInSession=4 ===
Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=Decimal('495.30729999999999790816218592226505279541015625'))


In [28]:
# 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
print("\n\n=== QUERY 2: Get artist, song, user info for userId=10, sessionId=182 (sorted by itemInSession) ===")
execute_query(
    session=session,
    query='''
        SELECT
            artist,
            song,
            firstname,
            lastname
        FROM
            udacity.user_sessions_songs
        WHERE
            userid = 10
            AND sessionid = 182
        ORDER BY
            iteminsession;
    ''',
    fetch=True
    )



=== QUERY 2: Get artist, song, user info for userId=10, sessionId=182 (sorted by itemInSession) ===
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [29]:
# 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
print("\n\n=== QUERY 3: Get all users who listened to 'All Hands Against His Own' ===")
execute_query(
    session=session,
    query='''
        SELECT
            userId,
            song,
            firstname,
            lastname
        FROM
            udacity.song_listeners
        WHERE
            song = 'All Hands Against His Own';
    ''',
    fetch=True
    )



=== QUERY 3: Get all users who listened to 'All Hands Against His Own' ===
Row(userid=29, song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(userid=80, song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(userid=95, song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [30]:
execute_query(session=session,query='DROP TABLE IF EXISTS udacity.sessions_songs;')

execute_query(session=session,query='DROP TABLE IF EXISTS udacity.user_sessions_songs;')

execute_query(session=session,query='DROP TABLE IF EXISTS udacity.song_listeners;')

### Close the session and cluster connection¶

In [96]:
session.shutdown()
cluster.shutdown()